<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Samples-table" data-toc-modified-id="Samples-table-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Samples table</a></span></li><li><span><a href="#Re-labeling-samples" data-toc-modified-id="Re-labeling-samples-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Re-labeling samples</a></span><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Format" data-toc-modified-id="Format-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Format</a></span></li><li><span><a href="#Write" data-toc-modified-id="Write-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Write</a></span></li></ul></li><li><span><a href="#LLA" data-toc-modified-id="LLA-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>LLA</a></span><ul class="toc-item"><li><span><a href="#Config" data-toc-modified-id="Config-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Config</a></span></li><li><span><a href="#Run" data-toc-modified-id="Run-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Run</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Summary</a></span></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* LLA on MiSeq Run116

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/Run0116/'

# metadata
metadata_file = '/ebio/abt3_projects/Georg_animal_feces/data/mapping/unified_metadata_complete_190529.tsv'

# for re-labeling
D = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Run0116/LLA/'
# base samples table used LLA; samples = MiSeq runIDs
samples_file = file.path(D, 'MiSeq-Run0116_samples.txt')
# Illumina samples sheet used by Iris
samples_idx_file = file.path(D, 'MiSeq-Run0116_Illumina.txt')

# params
pipeline_dir = '/ebio/abt3_projects/methanogen_host_evo/bin/lla'

# Init

In [3]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(LeyLabRMisc)

In [6]:
df.dims()

# Samples table

* Created from [local SRA](http://readarchive.eb.local/sra/)
  * MiSeq-run = 0116
* converted to LLA input samples table with `convert_SRA_to_samples_table.py`

# Re-labeling samples

* Iris used numbers instead of actual sampleIDs for the sample demux table
* Converting the number-based labels to the actual sampleIDs

## Load

In [7]:
meta = read.delim(metadata_file, sep='\t')
meta 

SampleID,host_subject_id,host_german_name,host_common_name,scientific_name,scientific_name_subspecies,host_taxid,class,order,family,⋯,temp_regulation,collection_date,season,samp_store_days,samp_collect_device,sampling_method,diet,feeding_style,digestive_system,digestive_system_detail
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
F14_Common_Bream,F14,Brachse,Common.Bream,Abramis_brama,NA,38527,Actinopterygii,Cypriniformes,Cyprinidae,⋯,ectotherm/poikilotherm,2010-08-10,summer,317,Greiner Polypropylene Tube 50ml,feces sampling,omnivore,NA,simple,simple
F35_Red_Deer,F35,Rothirsch,Red.Deer,Cervus_elaphus,NA,9860,Mammalia,Artiodactyla,Cervidae,⋯,endotherm/homoiotherm,2010-09-29,autumn,229,Greiner Polystyrene 30ml Faeces Container,feces sampling,herbivore,intermediate type,foregut,foregut
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X189_fail_Rainbow_Trout,189.fail,Regenbogenforelle,Rainbow.Trout,Oncorhynchus_mykiss,NA,8022,Actinopterygii,Salmoniformes,Salmonidae,⋯,ectotherm/poikilotherm,2012-03-01,winter,467,Eppendorf tube,feces sampling,carnivore,NA,simple,simple
X232_fail_Brown_Trout,232.fail,Seeforelle,Brown.Trout,Salmo_trutta,NA,8032,Actinopterygii,Salmoniformes,Salmonidae,⋯,ectotherm/poikilotherm,2012-03-15,winter,453,Polyethylene freezer bag,gut section,carnivore,NA,simple,simple


In [8]:
samp = read.delim(samples_file, sep='\t') %>%
    mutate(Sample_ID = gsub(' $', '', Sample))
samp

Sample,Run,Lane,Read1,Read2,Sample_ID
<fct>,<int>,<int>,<fct>,<fct>,<chr>
1,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId1_RunId0116_LaneId1/1_TAAGGCGA-TAGATCGC_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId1_RunId0116_LaneId1/1_TAAGGCGA-TAGATCGC_L001_R2_001.fastq.gz,1
101,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId101_RunId0116_LaneId1/101_CGTACTAG-GTAAGGAG_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId101_RunId0116_LaneId1/101_CGTACTAG-GTAAGGAG_L001_R2_001.fastq.gz,101
⋮,⋮,⋮,⋮,⋮,⋮
mock,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdmock_RunId0116_LaneId1/mock_CTCTCTAC-CTCTCTAT_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdmock_RunId0116_LaneId1/mock_CTCTCTAC-CTCTCTAT_L001_R2_001.fastq.gz,mock
water,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdwater_RunId0116_LaneId1/water_CTCTCTAC-TAGATCGC_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdwater_RunId0116_LaneId1/water_CTCTCTAC-TAGATCGC_L001_R2_001.fastq.gz,water


In [9]:
idx = read.delim(samples_idx_file, sep='\t') %>%
    mutate(Sample_ID = gsub(' $', '', Sample_ID))
idx

Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description
<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1.Eurasian.Beaver,Georg_PCR2_plate1,A01,N701,TAAGGCGA,S501,TAGATCGC,Amplicon,NickYoungblut
2,2.Raccoon.Dog,Georg_PCR2_plate1,B01,N701,TAAGGCGA,S502,CTCTCTAT,Amplicon,NickYoungblut
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
400,400.House.Mouse,Georg_PCR2_plate3,B10,N710,CGAGGCTG,S510,CCTCAGAC,Amplicon,NickYoungblut
Neg9,Neg.control.9,Georg_PCR2_plate3,C10,N710,CGAGGCTG,S511,TCCTTACG,Amplicon,NickYoungblut


## Format

In [10]:
# key overlap
setdiff(samp$Sample, idx$Sample_ID) %>% print
setdiff(idx$Sample_ID, samp$Sample) %>% print

character(0)
character(0)


In [11]:
# joining
samp = samp %>%
    inner_join(idx %>% dplyr::select(Sample_ID, Sample_Name), 
               c('Sample'='Sample_ID')) %>%
    rename('Illumina_RunID' = Sample,
           'Sample' = Sample_Name) %>%
    dplyr::select(Sample, Illumina_RunID, Run, Lane, Read1, Read2)

samp

Warning message:
“Column `Sample`/`Sample_ID` joining factor and character vector, coercing into character vector”


Sample,Illumina_RunID,Run,Lane,Read1,Read2
<fct>,<chr>,<int>,<int>,<fct>,<fct>
1.Eurasian.Beaver,1,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId1_RunId0116_LaneId1/1_TAAGGCGA-TAGATCGC_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId1_RunId0116_LaneId1/1_TAAGGCGA-TAGATCGC_L001_R2_001.fastq.gz
101.Horse,101,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId101_RunId0116_LaneId1/101_CGTACTAG-GTAAGGAG_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId101_RunId0116_LaneId1/101_CGTACTAG-GTAAGGAG_L001_R2_001.fastq.gz
⋮,⋮,⋮,⋮,⋮,⋮
mock,mock,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdmock_RunId0116_LaneId1/mock_CTCTCTAC-CTCTCTAT_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdmock_RunId0116_LaneId1/mock_CTCTCTAC-CTCTCTAT_L001_R2_001.fastq.gz
water,water,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdwater_RunId0116_LaneId1/water_CTCTCTAC-TAGATCGC_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdwater_RunId0116_LaneId1/water_CTCTCTAC-TAGATCGC_L001_R2_001.fastq.gz


In [12]:
# renaming to match meta
samp = samp %>%
    mutate(Sample = gsub('\\.', '_', Sample),
           Sample = gsub('^([0-9])', 'X\\1', Sample),
           Sample = gsub('(_control_[0-9]+)', '\\1X', Sample),
           Sample = gsub(' +$', '', Sample))

setdiff(samp$Sample, meta$SampleID)

[1] "X9_Indian_Gazelle" "Neg_control_1X"    "Neg_control_2X"   
 [4] "Neg_control_3X"    "Neg_control_4X"    "Neg_control_5X"   
 [7] "Neg_control_6X"    "Neg_control_7X"    "Neg_control_8X"   
[10] "Neg_control_9X"    "Pos_control_1X"    "Pos_control_2X"   
[13] "Pos_control_3X"    "Pos_control_4X"    "Pos_control_5X"   
[16] "Pos_control_6X"    "Pos_control_7X"    "Pos_control_8X"   
[19] "Pos_control_9X"    "methanogen"        "mock"             
[22] "water"

In [13]:
# filtering
samp = samp %>%
    filter(Sample != 'X9_Indian_Gazelle')

samp

Sample,Illumina_RunID,Run,Lane,Read1,Read2
<chr>,<chr>,<int>,<int>,<fct>,<fct>
X1_Eurasian_Beaver,1,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId1_RunId0116_LaneId1/1_TAAGGCGA-TAGATCGC_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId1_RunId0116_LaneId1/1_TAAGGCGA-TAGATCGC_L001_R2_001.fastq.gz
X101_Horse,101,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId101_RunId0116_LaneId1/101_CGTACTAG-GTAAGGAG_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleId101_RunId0116_LaneId1/101_CGTACTAG-GTAAGGAG_L001_R2_001.fastq.gz
⋮,⋮,⋮,⋮,⋮,⋮
mock,mock,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdmock_RunId0116_LaneId1/mock_CTCTCTAC-CTCTCTAT_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdmock_RunId0116_LaneId1/mock_CTCTCTAC-CTCTCTAT_L001_R2_001.fastq.gz
water,water,116,1,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdwater_RunId0116_LaneId1/water_CTCTCTAC-TAGATCGC_L001_R1_001.fastq.gz,/ebio/abt6_sra/years/2018/03_04/Nextera/illumina_M00333_flowcellA_SampleIdwater_RunId0116_LaneId1/water_CTCTCTAC-TAGATCGC_L001_R2_001.fastq.gz


## Write

In [14]:
out_file = file.path(work_dir, 'samples.txt')
write.table(samp, file=out_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', out_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//samples.txt 


# LLA

## Config

In [15]:
cat_file(file.path(work_dir, 'config.yaml'))

# table with sample --> read_file information 
samples_file: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116/samples.txt

# output location
output_dir: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116/


# taxonomy classifier database (SILVA by default)
classifier: /ebio/abt3_projects/databases/leylab16s/classifiers/scikit_0.21.2/silva-132-99-515-806-nb-classifier.qza

# params (change as needed)
params:
  trimmomatic:
    sample_size: 10000
    window_size: 20
    qlty_threshold: 25
  feature_table_merge:
    p_overlap_method: sum
  classify_sklearn:
    p_reads_per_batch: 1000
  filter_table:
    p_include:
      - D_1__
    p_exclude:
      - D_0__Eukaryota,D_4__mitochondria,D_2__Chloroplast


# snakemake pipeline
pipeline:
  snakemake_folder: ./
  script_folder: ./bin/scripts/
  temp_folder: /tmp/global2/nyoungblut/

## Run

```
(snakemake_dev) @ rick:/ebio/abt3_projects/methanogen_host_evo/bin/lla
$ screen -L -S LLA-run116 ./snakemake_sge.sh /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116/config.yaml cluster.json 30
```

# Summary

In [17]:
df.dims(20,20)
list.files(work_dir, full.names=TRUE) %>%
    as.list %>%
    lapply(file.info) %>%
    do.call(rbind, .)
df.dims()

,size,isdir,mode,mtime,ctime,atime,uid,gid,uname,grname
,<dbl>,<lgl>,<octmode>,<dttm>,<dttm>,<dttm>,<int>,<int>,<chr>,<chr>
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//aligned-rep-seqs.qza,5792978,FALSE,0644,2019-12-06 02:22:22,2019-12-06 02:22:22,2019-12-06 02:22:22,6354,353,nyoungblut,abt3
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//benchmark,4096,TRUE,2755,2019-12-06 03:13:35,2019-12-06 03:13:35,2019-12-04 17:13:45,6354,353,nyoungblut,abt3
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//config.yaml,919,FALSE,0644,2019-12-04 16:49:52,2019-12-04 16:49:52,2019-12-04 16:49:27,6354,353,nyoungblut,abt3
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//log,4096,TRUE,2755,2019-12-06 03:13:09,2019-12-06 03:13:09,2019-12-04 17:12:36,6354,353,nyoungblut,abt3
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//masked-aligned-rep-seqs.qza,5473103,FALSE,0644,2019-12-06 02:42:47,2019-12-06 02:42:47,2019-12-06 02:42:47,6354,353,nyoungblut,abt3
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//midpoint-rooted-tree.qza,1923959,FALSE,0644,2019-12-06 03:14:25,2019-12-06 03:14:25,2019-12-06 03:14:25,6354,353,nyoungblut,abt3
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//rep-seqs_merged_filt.qza,4084974,FALSE,0644,2019-12-06 01:35:36,2019-12-06 01:35:36,2019-12-06 01:35:36,6354,353,nyoungblut,abt3
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//rep-seqs_merged.qza,4738267,FALSE,0644,2019-12-06 00:29:51,2019-12-06 00:29:51,2019-12-06 00:29:51,6354,353,nyoungblut,abt3
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189/LLA/Run0116//Run116,216,TRUE,2755,2019-12-06 00:27:06,2019-12-06 00:27:06,2019-12-04 16:55:20,6354,353,nyoungblut,abt3


# sessionInfo

In [18]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/tidyverse/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.0 ggplot2_3.2.1     tidyr_1.0.0       dplyr_0.8.3      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.3       magrittr_1.5     munsell_0.5.0    tidyselect_0.2.5
 [5] uuid_0.1-2       colorspace_1.4-1 R6_2.4.1         rlang_0.4.2     
 [9] tools_3.6.2

In [19]:
pipelineInfo(pipeline_dir)

LLA
===

Ley Lab Amplicon pipeline (LLA)

* Version: 0.2.6
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
  * Guillermo Luque <guillermo.luque@tuebingen.mpg.de>
  * Jacobo de la Cuesta <jacobo.delacuesta@tuebingen.mpg.de>

--- conda envs ---
==> /ebio/abt3_projects/methanogen_host_evo/bin/lla/bin/envs/qiime2.yaml <==
channels:
- qiime2/label/r2019.10
- conda-forge
- bioconda
- defaults
dependencies:
- _libgcc_mutex=0.1
- _r-mutex=1.0.1
- alsa-lib=1.1.5
- arb-bio-tools=6.0.6
- asn1crypto=1.2.0
- atomicwrites=1.3.0
- attrs=19.3.0
- backcall=0.1.0
- bibtexparser=1.1.0
- binutils_impl_linux-64=2.31.1
- binutils_linux-64=2.31.1
- bioconductor-biobase=2.42.0
- bioconductor-biocgenerics=0.28.0
- bioconductor-biocparallel=1.16.6
- bioconductor-biostrings=2.50.2
- bioconductor-dada2=1.10.0
- bioconductor-delayedarray=0.8.0
- bioconductor-genomeinfodb=1.18.1
- bioconductor-genomeinfodbdata=1.2.1
- bioconductor-genomicalignments=1.18.1
- bioconductor-genomicranges=1.34.0
- bioconductor-irange